# Test coverage investigation
In this notebook, we analyze the data on what parameters are selected during various stages
of lowering by `./get_test_metedata.py`.

We aim to understand gaps in our test coverage and to identify over-represented code paths.

This notebook was composed under Python 3.10. Other notable version numbers include Numpy 1.22, Pandas 1.4, and Jupyter >= 4.7

In [1]:
import numpy as np
import pandas as pd

import pickle
from typing import Iterable

In [2]:
METADATA_FILE = "test_metadata_2022_04_28.pickle"

In [3]:
with open(METADATA_FILE, 'rb') as f:
    raw_xdlops, raw_non_xdlops = pickle.load(f)

In [4]:
raw_xdlops[0]

{'Config': '--operation conv2d_bwd_weight -t f32 -p=false -fil_layout=gkcyx -in_layout=ngchw -out_layout=ngkhw -batchsize=256 -groupsize=1 -in_channels=3 -out_channels=64 -in_h=224 -in_w=224 -fil_h=7 -fil_w=7 --dilation_h=1 --dilation_w=1 --padding_h=3 --padding_w=3 --conv_stride_h=2 --conv_stride_w=2',
 'Filename': PosixPath('../../mlir/test/rocmlir-driver/auto_e2e/padding_kernel_gemmN.mlir'),
 'Op': 'wrw',
 'Filter layout': 'gkcyx',
 'Input layout': 'ngchw',
 'Output layout': 'ngkhw',
 'g': 1,
 'k': 64,
 'c': 3,
 'y': 7,
 'x': 7,
 'n': 256,
 'hi': 224,
 'wi': 224,
 'ho': 112,
 'wo': 112,
 'Pad h left': 3,
 'Pad h right': 2,
 'Pad w left': 3,
 'Pad w right': 2,
 'Stride h': 2,
 'Stride w': 2,
 'Dilation h': 1,
 'Dilation w': 1,
 'Pad M': 0,
 'Pad K': 0,
 'Pad N': 45,
 'Block size': 64,
 'Grid size': 6,
 'M': 64,
 'K': 3211264,
 'N': 192,
 'M per block': 32,
 'K per block': 4,
 'N per block': 64,
 'KPack': 1,
 'M per wave': 32,
 'N per wave': 64,
 'Load A vector dim': 1,
 'Load B vecto

In [5]:
def to_touched_up_df(data) -> pd.DataFrame:
    ret = pd.DataFrame(data)
    for col in ["Load type", "Op", "Filter layout", "Input layout", "Output layout"]:
        ret[col] = ret[col].astype("category")
    ret["Filename"] = ret["Filename"].astype(str)
    ret["Filename"] = ret["Filename"].str.replace("../", "", regex=False)\
        .str.replace("mlir/test/rocmlir-driver/", "", regex=False)
    return ret

In [6]:
xdlops = to_touched_up_df(raw_xdlops)
non_xdlops = to_touched_up_df(raw_non_xdlops)

In [7]:
xdlops.head()

,Config,Filename,Op,Filter layout,Input layout,Output layout,g,k,c,y,...,Load A vector len,Load type,Load B vector len,Load A intermediate len,Load B intermediate len,LDS store A len,LDS store B len,Store C vector dim,Store C vector len,Xdlop
0,--operation conv2d_bwd_weight -t f32 -p=false ...,auto_e2e/padding_kernel_gemmN.mlir,wrw,gkcyx,ngchw,ngkhw,1,64,3,7,...,1,f32,1,2,4,1,1,4,1,mfma_f32_32x32x1f32
1,--operation conv2d_bwd_weight -t f16 -p=false ...,auto_e2e/padding_kernel_gemmN.mlir,wrw,gkcyx,ngchw,ngkhw,1,64,3,7,...,1,f16,1,2,4,1,1,4,1,mfma_f32_32x32x4f16
2,--operation conv2d_bwd_weight -t f32 -p=false ...,auto_e2e/padding_kernel_gemmN.mlir,wrw,gkyxc,nhwgc,nhwgk,1,64,3,7,...,1,f32,1,2,4,1,1,4,1,mfma_f32_32x32x1f32
3,--operation conv2d_bwd_weight -t f16 -p=false ...,auto_e2e/padding_kernel_gemmN.mlir,wrw,gkyxc,nhwgc,nhwgk,1,64,3,7,...,1,f16,1,2,4,1,1,4,1,mfma_f32_32x32x4f16
4,-fil_layout=yxck -in_layout=nhwc -out_layout=n...,auto_e2e/conv2d_host_validation_f32_fwd.mlir,fwd,gyxck,nhwgc,nhwgk,1,1024,1024,1,...,2,f32,2,8,8,1,1,3,4,mfma_f32_32x32x1f32


In [8]:
xdlops.shape

(662, 50)

In [9]:
non_xdlops.shape

(655, 47)

In [10]:
non_inherently_unique_cols_xdlops = xdlops.columns.drop(["Config", "Filename"])
non_inherently_unique_cols_xdlops

Index(['Op', 'Filter layout', 'Input layout', 'Output layout', 'g', 'k', 'c',
       'y', 'x', 'n', 'hi', 'wi', 'ho', 'wo', 'Pad h left', 'Pad h right',
       'Pad w left', 'Pad w right', 'Stride h', 'Stride w', 'Dilation h',
       'Dilation w', 'Pad M', 'Pad K', 'Pad N', 'Block size', 'Grid size', 'M',
       'K', 'N', 'M per block', 'K per block', 'N per block', 'KPack',
       'M per wave', 'N per wave', 'Load A vector dim', 'Load B vector dim',
       'Load A vector len', 'Load type', 'Load B vector len',
       'Load A intermediate len', 'Load B intermediate len', 'LDS store A len',
       'LDS store B len', 'Store C vector dim', 'Store C vector len', 'Xdlop'],
      dtype='object')

In [11]:
non_inherently_unique_cols_non_xdlops = non_xdlops.columns.drop(["Config", "Filename"])
non_inherently_unique_cols_non_xdlops

Index(['Op', 'Filter layout', 'Input layout', 'Output layout', 'g', 'k', 'y',
       'x', 'c', 'n', 'hi', 'wi', 'ho', 'wo', 'Pad h left', 'Pad h right',
       'Pad w left', 'Pad w right', 'Stride h', 'Stride w', 'Dilation h',
       'Dilation w', 'Pad M', 'Pad K', 'Pad N', 'Block size', 'Grid size', 'M',
       'K', 'N', 'M per block', 'K per block', 'N per block', 'KPack',
       'M per thread', 'N per thread', 'Load A vector dim',
       'Load B vector dim', 'Load A vector len', 'Load type',
       'Load B vector len', 'Load A intermediate len',
       'Load B intermediate len', 'LDS store A len', 'LDS store B len'],
      dtype='object')

In [12]:
def duplicates(df: pd.DataFrame, cols_to_test) -> pd.DataFrame:
    return df[df.duplicated(subset=cols_to_test, keep=False)]

In [13]:
xdlops_dups = duplicates(xdlops, non_inherently_unique_cols_xdlops)
for conf, group in xdlops_dups.groupby(list(non_inherently_unique_cols_xdlops)):
    print("Shared:", dict(zip(non_inherently_unique_cols_xdlops, conf)))
    for _, row in group.iterrows():
        print("Config:", row["Config"])
        print("File:", row["Filename"])
    print()

Shared: {'Op': 'bwd', 'Filter layout': 'gkcyx', 'Input layout': 'ngchw', 'Output layout': 'ngkhw', 'g': 1, 'k': 128, 'c': 128, 'y': 3, 'x': 3, 'n': 64, 'hi': 28, 'wi': 28, 'ho': 28, 'wo': 28, 'Pad h left': 1, 'Pad h right': 1, 'Pad w left': 1, 'Pad w right': 1, 'Stride h': 1, 'Stride w': 1, 'Dilation h': 1, 'Dilation w': 1, 'Pad M': 0, 'Pad K': 0, 'Pad N': 0, 'Block size': 256, 'Grid size': 392, 'M': 128, 'K': 288, 'N': 50176, 'M per block': 128, 'K per block': 4, 'N per block': 128, 'KPack': 4, 'M per wave': 64, 'N per wave': 64, 'Load A vector dim': 2, 'Load B vector dim': 2, 'Load A vector len': 1, 'Load type': 'f32', 'Load B vector len': 1, 'Load A intermediate len': 8, 'Load B intermediate len': 8, 'LDS store A len': 1, 'LDS store B len': 1, 'Store C vector dim': -1, 'Store C vector len': 1, 'Xdlop': 'mfma_f32_32x32x1f32'}
Config: -batchsize=64 -in_channels=128 -in_h=28 -in_w=28 -out_channels=128 -fil_h=3 -fil_w=3 --dilation_h=1 --dilation_w=1 --conv_stride_h=1 --conv_stride_w=1 -

In [14]:
non_xdlops_dups = duplicates(non_xdlops, non_inherently_unique_cols_non_xdlops)
for conf, group in non_xdlops_dups.groupby(list(non_inherently_unique_cols_non_xdlops)):
    print("Shared:", dict(zip(non_inherently_unique_cols_non_xdlops, conf)))
    for _, row in group.iterrows():
        print("Config:", row["Config"])
        print("File:", row["Filename"])
    print()

Shared: {'Op': 'bwd', 'Filter layout': 'gkcyx', 'Input layout': 'ngchw', 'Output layout': 'ngkhw', 'g': 1, 'k': 128, 'y': 1, 'x': 1, 'c': 256, 'n': 64, 'hi': 56, 'wi': 56, 'ho': 56, 'wo': 56, 'Pad h left': 0, 'Pad h right': 0, 'Pad w left': 0, 'Pad w right': 0, 'Stride h': 1, 'Stride w': 1, 'Dilation h': 1, 'Dilation w': 1, 'Pad M': 0, 'Pad K': 0, 'Pad N': 0, 'Block size': 256, 'Grid size': 3136, 'M': 256, 'K': 128, 'N': 200704, 'M per block': 128, 'K per block': 16, 'N per block': 128, 'KPack': 1, 'M per thread': 4, 'N per thread': 4, 'Load A vector dim': 2, 'Load B vector dim': 2, 'Load A vector len': 1, 'Load type': 'f32', 'Load B vector len': 1, 'Load A intermediate len': 8, 'Load B intermediate len': 8, 'LDS store A len': 1, 'LDS store B len': 1}
Config: -batchsize=64 -groupsize=1 -in_channels=256 -in_h=56 -in_w=56 -out_channels=128 -fil_h=1 -fil_w=1 --dilation_h=1 --dilation_w=1 --conv_stride_h=1 --conv_stride_w=1 --padding_h=0 --padding_w=0 --operation conv2d_bwd_data -fil_layou

In [15]:
xdlops_dups.groupby("Filename").count()

,Config,Op,Filter layout,Input layout,Output layout,g,k,c,y,x,...,Load A vector len,Load type,Load B vector len,Load A intermediate len,Load B intermediate len,LDS store A len,LDS store B len,Store C vector dim,Store C vector len,Xdlop
Filename,,,,,,,,,,,,,,,,,,,,,
auto_e2e/Resnet50/F16_tests/Resnet50_f16_1.mlir,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
auto_e2e/Resnet50/F16_tests/Resnet50_f16_2.mlir,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
auto_e2e/Resnet50/F16_tests/Resnet50_f16_4.mlir,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
auto_e2e/Resnet50/F16_tests/Resnet50_f16_6.mlir,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
auto_e2e/Resnet50/F16_tests/Resnet50_f16_9.mlir,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
auto_e2e/Resnet50/F32_tests/Resnet50_f32_1.mlir,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
auto_e2e/Resnet50/F32_tests/Resnet50_f32_11.mlir,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
auto_e2e/Resnet50/F32_tests/Resnet50_f32_12.mlir,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
auto_e2e/Resnet50/F32_tests/Resnet50_f32_15.mlir,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [16]:
xdlops_dups.shape

(137, 50)

In [17]:
non_xdlops_dups.shape

(137, 47)

In [18]:
for conf, group in non_xdlops_dups.groupby(list(non_inherently_unique_cols_non_xdlops)):
    conf = dict(zip(non_inherently_unique_cols_non_xdlops, conf))
    if conf["Op"] != "fwd" or conf["Input layout"] != "ngchw":
        continue
    if not (group["Filename"].str.contains("Resnext").any() and\
            group["Filename"].str.contains("Resnet").any()):
        continue
    for _, row in group.iterrows():
        print("File:", row["Filename"])
    print()

File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_15.mlir
File: auto_e2e/Resnet50/F32_tests/Resnet50_f32_11.mlir

File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_22.mlir
File: auto_e2e/Resnet50/F32_tests/Resnet50_f32_20.mlir

File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_20.mlir
File: auto_e2e/Resnet50/F32_tests/Resnet50_f32_17.mlir

File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_17.mlir
File: auto_e2e/Resnet50/F32_tests/Resnet50_f32_12.mlir

File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_10.mlir
File: auto_e2e/Resnet50/F32_tests/Resnet50_f32_3.mlir

File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_19.mlir
File: auto_e2e/Resnet50/F32_tests/Resnet50_f32_15.mlir

File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_9.mlir
File: auto_e2e/Resnet50/F32_tests/Resnet50_f32_1.mlir



In [19]:
for conf, group in non_xdlops_dups.groupby(list(non_inherently_unique_cols_non_xdlops)):
    if not group["Filename"].str.contains("padding_kernel").any():
        continue
    for _, row in group.iterrows():
        print("Config:", row["Config"])
        print("File:", row["Filename"])
    print()

Config: --operation conv2d -t f32 -p=false -fil_layout=gkcyx -in_layout=ngchw -out_layout=ngkhw -batchsize=256 -groupsize=1 -in_channels=3 -out_channels=64 -in_h=224 -in_w=224 -fil_h=7 -fil_w=7 --dilation_h=1 --dilation_w=1 --padding_h=3 --padding_w=3 --conv_stride_h=2 --conv_stride_w=2
File: auto_e2e/padding_kernel_gemmK.mlir
Config: -fil_layout=gkcyx -in_layout=ngchw -out_layout=ngkhw -batchsize=256 -groupsize=1 -in_channels=3 -in_h=224 -in_w=224 -out_channels=64 -fil_h=7 -fil_w=7 --dilation_h=1 --dilation_w=1 --conv_stride_h=2 --conv_stride_w=2 --padding_h=3 --padding_w=3 -t f32 --operation=conv2d
File: auto_e2e/Resnext101/F32_tests/Resnext101_f32_23.mlir

Config: --operation conv2d -t f32 -p=false -fil_layout=gkyxc -in_layout=nhwgc -out_layout=nhwgk -batchsize=256 -groupsize=1 -in_channels=3 -out_channels=64 -in_h=224 -in_w=224 -fil_h=7 -fil_w=7 --dilation_h=1 --dilation_w=1 --padding_h=3 --padding_w=3 --conv_stride_h=2 --conv_stride_w=2
File: auto_e2e/padding_kernel_gemmK.mlir
Con

In [20]:
for conf, group in non_xdlops_dups.groupby(list(non_inherently_unique_cols_non_xdlops)):
    if not group["Filename"].str.contains("host_validation").any():
        continue
    for _, row in group.iterrows():
        print("Config:", row["Config"])
        print("File:", row["Filename"])
    print()

Config: --rand_type float --operation conv2d -t f16 --fil_layout kcyx --in_layout nchw --out_layout nkhw --batchsize 256 --in_channels 128 --in_h 58 --in_w 58 --out_channels 128 --fil_w 3 --fil_h 3 --dilation_h 1 --dilation_w 1 --conv_stride_h 2 --conv_stride_w 2 --padding_h 0 --padding_w 0
File: auto_e2e/conv2d_host_validation_f16_fwd.mlir
Config: --rand_type float --operation conv2d -t f16 --fil_layout kcyx --in_layout nchw --out_layout nkhw --batchsize 256 --in_channels 128 --in_h 58 --in_w 58 --out_channels 128 --fil_w 3 --fil_h 3 --dilation_h 1 --dilation_w 1 --conv_stride_h 2 --conv_stride_w 2 --padding_h 0 --padding_w 0
File: auto_e2e/conv2d_host_validation_f16_fwd.mlir

Config: --rand_type float --operation conv2d -t f16 --fil_layout kcyx --in_layout nchw --out_layout nkhw --batchsize 256 --in_channels 256 --in_h 14 --in_w 14 --out_channels 256 --fil_w 3 --fil_h 3 --dilation_h 1 --dilation_w 1 --conv_stride_h 1 --conv_stride_w 1 --padding_h 1 --padding_w 1
File: auto_e2e/conv2d

In [21]:
## TODO: when rerunning this, account for rand-type differences